### Moved a lot to snakemake for speed, memory and readabilitiy.

In [1]:
vcf = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/variants.all.vcf"
bam_fs = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc"
outdir = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap"
genome_fa = "/data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist_A2_2020/fasta/genome.fa"
scripts_dir = "/data/Mito_Trace/src/mtpreproc/"

In [2]:
from glob import glob
from os.path import basename, join, exists
import os
from src.utils.data_io import read_csv_multichar
import pandas as pd

fig_utils from mplh


In [3]:
regions_outdir = join(outdir, "regions")
if not exists(regions_outdir):
    os.mkdir(regions_outdir)
    
# sc_regions_outdir = join(regions_outdir, "sc")
# if not exists(sc_regions_outdir):
#     os.mkdir(sc_regions_outdir)

In [4]:
b = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam"


In [6]:
#for b in glob(bam_fs+"/*.bam"):
#    cmd = f"bedtools intersect -wa -a {b} -b {vcf}  > {outdir}/{basename(b)}"
#    print(cmd)
#    !{cmd}
#    cmd = f"samtools index {outdir}/{basename(b)}"
#    print(cmd)
#    !{cmd}
if not exists(join(outdir,basename(b)+".bai")):
    cmd = f"samtools index {outdir}/{basename(b)}"
    print(cmd)
    !{cmd}

In [8]:
curr_b = f"{outdir}/{basename(b)}"
curr_b_outdir = f'{curr_b.replace(".bam", "")}_pileups'
if not exists(curr_b_outdir):
    os.mkdir(curr_b_outdir)
cmd = f"python {scripts_dir}/split_by_CB.py {curr_b} {curr_b_outdir}"
print(cmd)
!{cmd}

python /data/Mito_Trace/src/mtpreproc//split_by_CB.py /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input.bam /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input_pileups
/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input.bam /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input.CB.bam
samtools sort -t CB /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input.bam > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b1_Input.CB.bam
[bam_sort_core] merging from 80 files and 1 in-memory blocks...
150133000it [44:57, 55655.99it/s] 
no CBs:  8487066
no CBs permission 0


## Bedtools intersect for each specific region
### Then add each variant for counts and variants

In [6]:
region_f = vcf+".region.txt"
vcf_df = read_csv_multichar(vcf, sep="\t", multicomment="##")
vcf_df[["#CHROM", "POS"]].to_csv(region_f, index=None, sep="\t", header=None)

In [7]:
vcf_df = vcf_df.loc[vcf_df["#CHROM"].str.contains("^chr")]
vcf_df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,CHIP_b1_Control,...,CHIP_b1_Input,CHIP_b2_Control,CHIP_b2_Flt3l,CHIP_b2_Input,CHIP_b3_Control,CHIP_b3_Flt3l,CHIP_b3_Input,CHIP_b4_Control,CHIP_b4_Flt3l,CHIP_b4_Input
1000,chr1,10018,.,C,G,97.47380,.,TYPE=snv;NS=12;AF=0.08333333;DP=4744;RO=4722;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0.2387064:681:678:1:0.001468429:402,276,1,...",...,"0/1:97.4738:1249:1237:12:0.009607686:740,497,7...","0/0:0:234:233:0:0:113,120,0,0:-1:-1:0:944.2852:.","0/0:0:225:225:0:0:113,112,0,0:-1:-1:0:926.5604:.","0/0:0:343:343:0:0:176,167,0,0:-1:-1:0:1432.95:.","0/0:16.98098:186:184:2:0.01075269:100,84,0,2:9...","0/0:0:269:269:0:0:130,139,0,0:-1:-1:0:1116.99:.","0/0:0:108:108:0:0:53,55,0,0:-1:-1:0:416.3386:.","0/0:21.71786:356:353:3:0.008426966:162,191,2,1...","0/0:0:315:314:0:0:160,154,0,0:-1:-1:0:1305.324:.","0/0:0:118:118:0:0:67,51,0,0:-1:-1:0:458.7915:."
1001,chr1,10031,.,T,TA,65.50475,.,TYPE=ins;NS=12;AF=0.08333333;DP=6095;RO=6077;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:65.50475:856:853:5:0.005841121:532,321,3,2...",...,"0/0:0:1640:1639:0:0:1029,610,0,0:-1:-1:0:Inf:.","0/0:0:315:315:0:0:163,152,0,0:-1:-1:0:1790.363:.","0/0:0:286:286:0:0:146,140,0,0:-1:-1:0:1616.495:.","0/0:0:432:431:0:0:237,194,0,0:-1:-1:0:2483.383:.","0/0:0:245:245:0:0:141,104,0,0:-1:-1:0:1386.389:.","0/0:0:357:357:0:0:189,168,0,0:-1:-1:0:2052.156:.","0/0:0:141:141:0:0:70,71,0,0:-1:-1:0:744.5862:.","0/0:0:446:444:0:0:230,214,0,0:-1:-1:0:2570.377:.","0/0:0:381:379:0:0:208,171,0,0:-1:-1:0:2196.873:.","0/0:0:154:153:0:0:94,59,0,0:-1:-1:0:833.2017:."
1002,chr1,10039,.,AC,A,87.23672,.,TYPE=del;NS=12;AF=0.08333333;DP=6678;RO=6620;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:946:939:0:0:633,306,0,0:-1:-1:0:Inf:.",...,"0/1:87.23672:1822:1808:8:0.004390779:1257,551,...","0/0:0:341:340:0:0:195,145,0,0:-1:-1:0:1873.462:.","0/0:0:308:306:0:0:178,128,0,0:-1:-1:0:1679.652:.","0/0:0:472:470:0:0:277,193,0,0:-1:-1:0:2615.803:.","0/0:0:259:259:0:0:168,91,0,0:-1:-1:0:1403.992:.","0/0:22.05355:368:361:2:0.005434783:218,143,0,2...","0/0:0:161:160:0:0:94,66,0,0:-1:-1:0:819.4477:.","0/0:21.33713:492:489:2:0.004065041:288,201,2,0...","0/0:0:422:422:0:0:251,171,0,0:-1:-1:0:2310.822:.","0/0:0:176:174:0:0:124,50,0,0:-1:-1:0:933.9806:."
1003,chr1,10051,.,A,ACCG,67.72425,.,TYPE=ins;NS=12;AF=0.08333333;DP=7730;RO=7680;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:0:1123:1117:0:0:754,363,0,0:-1:-1:0:Inf:.",...,"0/1:67.72425:2162:2147:6:0.002775208:1497,650,...","0/0:0:384:382:0:0:230,152,0,0:-1:-1:0:2286.389:.","0/0:0:348:348:0:0:212,136,0,0:-1:-1:0:2077.343:.","0/0:0:535:532:0:0:317,215,0,0:-1:-1:0:Inf:.","0/0:0:293:291:0:0:191,100,0,0:-1:-1:0:1747.041:.","0/0:0:419:416:0:0:272,144,0,0:-1:-1:0:2530.343:.","0/0:0:190:190:0:0:113,77,0,0:-1:-1:0:1078.799:.","0/0:0:562:561:0:0:346,215,0,0:-1:-1:0:Inf:.","0/0:0:456:456:0:0:289,167,0,0:-1:-1:0:2738.056:.","0/0:0:208:206:0:0:137,69,0,0:-1:-1:0:1201.061:."
1004,chr1,10055,.,TA,T,60.01402,.,TYPE=del;NS=12;AF=0.1666667;DP=7734;RO=7677;AO...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/1:60.01402:1122:1113:9:0.00802139:707,406,4,...",...,"0/0:0:2135:2124:1:0.0004683841:1442,682,1,0:99...","0/0:0:389:388:0:0:236,152,0,0:-1:-1:0:1570.522:.","0/0:0:352:352:0:0:216,136,0,0:-1:-1:0:1430.16:.","0/0:0.5660855:535:535:1:0.001869159:314,221,0,...","0/0:0:295:294:0:0:196,98,0,0:-1:-1:0:1183.517:.","0/0:0.5660855:422:421:1:0.002369668:270,151,0,...","0/0:0:189:186:0:0:107,79,0,0:-1:-1:0:730.9136:.","0/0:0:557:555:0:0:335,220,0,0:-1:-1:0:2283.826:.","0/0:0:451:449:0:0:288,161,0,0:-1:-1:0:1836.398:.","0/0:0:198:197:0:0:128,69,0,0:-1:-1:0:772.6831:."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6200,chrY,56767254,.,T,A,56.78491,.,TYPE=snv;NS=12;AF=0.08333333;DP=1759;RO=1744;A...,GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:...,"0/0:20.33698:408:405:3:0.007352941:277,128,0,3...",...,"0/0:0:380:380:0:0:252,128,0,0:-1:-1:0

In [9]:
for ind, region in vcf_df.iterrows(): 
    #for b in glob(bam_fs+"/*.bam"):
    region_str = f"{region['#CHROM']}:{region['POS']}-{region['POS']}"
    region_f =  f"{region['#CHROM']}_{region['POS']}" #region_str.replace(":", "_").replace("-", "_")
    cmd = f"""samtools view -b {b} {region_str}  > {regions_outdir}/{region_f}_{basename(b)}"""
    print(cmd)
    !{cmd}

samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10018-10018  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10018_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10031-10031  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10031_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10039-10039  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10039_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10051-10051  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10051_CHIP_b1_Input.bam
samtools

samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10377-10377  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10377_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10377-10377  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10377_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10403-10403  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10403_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:10421-10421  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_10421_CHIP_b1_Input.bam
samtools

samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16514226-16514226  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16514226_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16514262-16514262  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16514262_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16514273-16514273  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16514273_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16644730-16644730  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1

samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16740507-16740507  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16740507_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16740536-16740536  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16740536_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16740562-16740562  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_16740562_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:16740575-16740575  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1

samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:23344514-23344514  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_23344514_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:23344522-23344522  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_23344522_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:23980384-23980384  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1_23980384_CHIP_b1_Input.bam
samtools view -b /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b1_Input.bam chr1:27490267-27490267  > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/regions/chr1

UnboundLocalError: local variable 'child' referenced before assignment

## Break-up bam into single-cell. Then do pileup for each. 
a. Sort by CB
b. Loop 

In [ ]:
### a. split_by_cb but add the read group or the id to the name (have a dictionary)
### b. run the pileup for snps

In [ ]:
#"samtools mpileup -f reference.fa -r chr22:425236-425236 alignments.bam | cut -f 5 | tr '[a-z]' '[A-Z]' | fold -w 1 | sort | uniq -c"

In [14]:
region_f = vcf+".region.txt"
vcf_df = read_csv_multichar(vcf, sep="\t", multicomment="##")
vcf_df[["#CHROM", "POS"]].to_csv(region_f, index=None, sep="\t", header=None)

## pileup for each cb and position

In [ ]:
# for ind, region in vcf_df.iterrows(): 
#     #for b in glob(bam_fs+"/*.bam"):
#     region_str = f"{region['#CHROM']}:{region['POS']}-{region['POS']}"
#     region_f =  f"{region['#CHROM']}_{region['POS']}" #region_str.replace(":", "_").replace("-", "_")
#     cmd = f"""samtools view -b {b} {region_str} | samtools fillmd -e - {genome_fa} | grep -v "^@"| awk -v pos="{region['POS']}" 'BEGIN {{OFS = FS = "\t" }} ; {{n=split($10,a,"") ; if(a[(pos-$4)+1] != "=" ) print pos,(pos-$4)+1, a[(pos-$4)+1], $1, $4, $10 }}'  > {outdir}/{basename(b)}_{region_f}"""
#     print(cmd)
#         #!{cmd}

In [26]:
cmd = f"samtools mpileup -t AD -f {genome_fa} -l {region_f} {b} > {join(outdir, basename(b)+'.pileup.txt')}"
print(cmd)

samtools mpileup -t AD -f /data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist_A2_2020/fasta/genome.fa -l /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/variants.all.vcf.region.txt /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/preproc/CHIP_b3_Control.bam > /data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/bam_vcf_overlap/CHIP_b3_Control.bam.pileup.txt


In [31]:
sampile = pd.read_csv(join(outdir, basename(b)+'.pileup.txt'), sep="\t", header=None)
sampile

,0,1,2,3,4,5
0,chr1,10018,C,186,",,,,,,,g,,.,.,,,,,,,,,,,,,,,,,,,,,,,,,,,g,,......",AjkFCC.;FQ\FkFFQFFFkkFFFFkFFFkkFFFkQkFF:>kk_kk...
1,chr1,10031,T,245,",,,,,,,,,..,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,......",kkkCC.;kk^<kiFkFkkFFFFk_FF_kFFkk_kFkFk_::k_Ekk...
2,chr1,10039,A,259,",,,,,,,,,,,,,,,,,,,,,,,+1c,,,,,,,,,,,....,,,,,...",kkCC.kkkikkFkQFk_Q:kkFFkkkkkFk_EkE>kk`BFF:F^:g...
3,chr1,10051,A,293,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,....,,,,.........",khCSkkk:k_kkk_FkkFF__kkQkkkk9]kkkkkFFF:^:gFFQF...
4,chr1,10055,T,295,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,.....,,...",Sk_kFk_kkkQFEk>FkQQkkQkkk9ikkkk`FF:kF:^gFkFFi:...
...,...,...,...,...,...,...
6045,KI270337.1,852,C,12,",$,.......,,.",DFFkFFFkFFFF
6046,KI270337.1,894,C,7,"...,,..",:FFFFF>
6047,KI270336.1,923,A,6,",,,,,.",k:Fkkk
6048,KI270336.1,937,A,6,",,,,,.",_FFkQk


In [27]:
!{cmd}

[warning] samtools mpileup option `t` is functional, but deprecated. Please switch to using bcftools mpileup in future.
[mpileup] 1 samples in 1 input files


In [ ]:
chr1    143215235       .       G       C       57.48579        .       TYPE=snv;NS=12;AF=0.08333333;DP=2830;RO=2787;AO=10;SRF=1557;SRR=1230;SAF=5;SAR=5;SOR=0.4849351;RVSB=0.5584555;FS=1.206317;ERR=0.0002171339;SIG=0.001;CONT=ATCxAAT       GT:QVAL:DP:RO:AO:AF:SB:SOR:RVSB:FS:QVAL_minAF:STATUS    0/1:57.48579:526:517:5:0.009505703:289,228,2,3:1.05515:0.6539326:1.809933:2665.46:.     0/0:30.6072:450:442:3:0.006666667:260,182,2,1:1.085189:0.5828025:0:2263.195:.   0/0:17.91017:427:417:2:0.004683841:243,174,1,1:0.4139138:0.5823389:0:2136.623:. 0/0:0:47:46:0:0:24,22,0,0:-1:-1:0:165.2151:.    ./.:0:23:23:0:0:13,10,0,0:-1:-1:0:38.25952:.    0/0:0:1083:1072:0:0:576,496,0,0:-1:-1:0:Inf:.   ./.:0:10:9:0:0:4,5,0,0:-1:-1:0:0:.      ./.:0:15:15:0:0:11,4,0,0:-1:-1:0:14.46496:.     0/0:0:55:55:0:0:29,26,0,0:-1:-1:0:217.4113:.    0/0:0:62:61:0:0:35,26,0,0:-1:-1:0:241.4051:.    0/0:0:77:75:0:0:45,30,0,0:-1:-1:0:317.5159:.    0/0:0:55:55:0:0:28,27,0,0:-1:-1:0:217.4113:.
chr1    143215344       .       G       C       78.56018        .       TYPE=snv;NS=12;AF=0.08333333;DP=2838;RO=2814;AO=12;SRF=996;SRR=1818;SAF=4;SAR=8;SOR=0.7887207;RVSB=0.5227392;FS=0;ERR=0.0002879004;SIG=0.00